In [103]:
from __future__ import print_function, division
import sys,os
# line 4 and line 5 below are for development purposes and can be removed
qspin_path = os.path.join(os.getcwd(),"../../")
sys.path.insert(0,qspin_path)
from scipy.linalg import expm
from quspin.operators import hamiltonian, commutator, exp_op # Hamiltonians and operators
from quspin.basis import tensor_basis,spinless_fermion_basis_1d,boson_basis_1d, spin_basis_1d # bases
from quspin.tools.measurements import obs_vs_time # calculating dynamics
from quspin.tools.Floquet import Floquet_t_vec # period-spaced time vector
import numpy as np # general math functions
import matplotlib.pyplot as plt # plotting library

In [104]:
L = 3
M = 0.1
l=1
no_checks = dict(check_pcon=False,check_symm=False,check_herm=False)

In [136]:
basis_spin =spin_basis_1d(L=L,S = str(l))
basis_fermions = spin_basis_1d(L=L+1,S="1/2")
basis=tensor_basis(basis_spin,basis_fermions)
print(basis)

reference states: 
array index   /   Fock state   /   integer repr. 
	  0.         |2 2 2>           26         |1 1 1 1>           15  
	  1.         |2 2 2>           26         |1 1 1 0>           14  
	  2.         |2 2 2>           26         |1 1 0 1>           13  
	  3.         |2 2 2>           26         |1 1 0 0>           12  
	  4.         |2 2 2>           26         |1 0 1 1>           11  
	  5.         |2 2 2>           26         |1 0 1 0>           10  
	  6.         |2 2 2>           26         |1 0 0 1>            9  
	  7.         |2 2 2>           26         |1 0 0 0>            8  
	  8.         |2 2 2>           26         |0 1 1 1>            7  
	  9.         |2 2 2>           26         |0 1 1 0>            6  
	 10.         |2 2 2>           26         |0 1 0 1>            5  
	 11.         |2 2 2>           26         |0 1 0 0>            4  
	 12.         |2 2 2>           26         |0 0 1 1>            3  
	 13.         |2 2 2>           26         |0 0

In [144]:
basis_vectors = []
for index in range(basis.Ns):
    state = np.zeros(basis.Ns)
    state[basis.Ns-index-1] = 1.
    state = state.tolist()
    basis_vectors.append(state)

In [145]:
gauss_law_z_0 = []
gauss_law_z_0.append([1.,0])

gauss_law_idz_0 = [[-0.5,0]]

gauss_law_f_0 = [[-0.5,0]]

gauss_law_0_map = [
    
    ["z|",gauss_law_z_0],
    ["|z",gauss_law_f_0],
    ["|I",gauss_law_idz_0],


    
]

Gauss_law_0  = hamiltonian(gauss_law_0_map,dynamic_list=[],basis=basis,**no_checks)
Gauss_0_q = Gauss_law_0**2

In [146]:
#Gauss_law_1
gauss_law_zm_1 = []
gauss_law_zm_1.append([-1.,0])

gauss_law_z_1 = []
gauss_law_z_1.append([1.,1])

gauss_law_id_1 = [[1.,1]]
gauss_law_f_1 = [[-0.5,1]]

gauss_law_idz_1 = [[-0.5,1]]

gauss_law_1_map = [
    
    ["z|",gauss_law_z_1],
    ["z|",gauss_law_zm_1],
    ["I|",gauss_law_id_1],
    ["|z",gauss_law_f_1],
    ["|I",gauss_law_idz_1],

]

Gauss_law_1  = hamiltonian(gauss_law_1_map,dynamic_list=[],basis=basis,**no_checks)
Gauss_1_q = Gauss_law_1**2

In [147]:
#Gauss_law_2
gauss_law_zm_2 = []
gauss_law_zm_2.append([-1.,1])

gauss_law_z_2 = []
gauss_law_z_2.append([1.,2])

gauss_law_idz_2 = [[-0.5,2]]


gauss_law_f_2 = [[-0.5,2]]
gauss_law_2_map = [
    
    ["z|",gauss_law_z_2],
    ["z|",gauss_law_zm_2],
    ["|z",gauss_law_f_2],
    ["|I",gauss_law_idz_2],


]

Gauss_law_2  = hamiltonian(gauss_law_2_map,dynamic_list=[],basis=basis,**no_checks)
Gauss_2_q = Gauss_law_2**2

In [148]:
#Gauss_law_3

gauss_law_z_3 = []
gauss_law_z_3.append([-1.,2])

gauss_law_id_3 = [[1.,2]]
gauss_law_f_3 = [[-0.5,3]]

gauss_law_idz_3 = [[-0.5,3]]


gauss_law_3_map = [
    
    ["z|",gauss_law_z_3],
    ["I|",gauss_law_id_3],
    ["|z",gauss_law_f_3],
    ["|I",gauss_law_idz_3],



    
]

Gauss_law_3  = hamiltonian(gauss_law_3_map,dynamic_list=[],basis=basis,**no_checks)
Gauss_3_q = Gauss_law_3**2

In [149]:
G = Gauss_0_q+Gauss_1_q+Gauss_2_q+Gauss_3_q

In [150]:
for i in range(basis.Ns):
    if (G.expt_value(basis_vectors[basis.Ns-i-1])<=0.+1e-8) & (G.expt_value(basis_vectors[basis.Ns-i-1])>=0.-1e-8):
        print(i)

3
53
70
201
218
268


In [127]:
kin_energy = [[0.5,i,i] for i in range(L)]
mass_term = [[0.5*M*(-1)**i,i] for i in range(L+1)]
interaction_term = [[-1j/(2*np.sqrt(2)),i,i,i+1] for i in range(L)]

hamiltonian_map = [
    
    ["zz|",kin_energy],
    ["|z",mass_term],
    ["|I",mass_term],

    ["+|+-",interaction_term],
    ["-|-+",interaction_term]
]



Ham_LGT = hamiltonian(hamiltonian_map,dynamic_list = [],basis=basis,**no_checks)
Hamiltonian_LGT = Ham_LGT + 1000*G

In [128]:
ham_matrix = Hamiltonian_LGT.toarray()

In [129]:
def check_hermitian(a, rtol=1e-10, atol=1e-10):
    return np.allclose(a, a.conj().T, rtol=rtol, atol=atol)

In [130]:
print(check_hermitian(ham_matrix))

True


In [131]:
eigenval,eigenvec = np.linalg.eigh(ham_matrix)

In [132]:
for i in range(len(eigenval)):
    if eigenval[i]<=100:
        print(eigenval[i])
        print(i)

-0.8455138463653484
0
-0.5656905202659082
1
0.09336559193286807
2
0.22748942052892093
3
0.33663195365040993
4
0.5000000000009547
5
0.7769249994146044
6
0.8672216647797004
7
0.9658903438802167
8
1.0000000000003215
9
1.2975418340556117
10
1.36183690183109
11
1.8407440641877286
12
2.043557592363562
13
